In [165]:
import datetime as dt
import requests
import json
import pandas as pd
import ccxt
import plotly.graph_objects as px
from plotly.subplots import make_subplots

In [166]:
symbol_name = "LINK"

cg_headers = {'coinglassSecret': '7ef635a6899a4fbdb9db368d222c974d'}
candles = {"1m": 9, "5m": 3, "15m": 10, "30m": 11, "1h": 2, "4h": 1, "12h": 4, "24h": 5, "90d": 18, "all": 0}


In [167]:
# 定義資料取得方法
def get_oi_data(symbol, Timeframe="1m"):
    timeType = candles[Timeframe]
    url = f"https://open-api.coinglass.com/api/pro/v1/futures/openInterest?symbol={symbol}&interval={timeType}"
    response = requests.request("GET", url, headers=cg_headers)
    return json.loads(response.text)
    
def get_oi_chart_data(symbol, Timeframe):
    timeType = candles[Timeframe]
    url = f"https://open-api.coinglass.com/api/pro/v1/futures/openInterest/chart?symbol={symbol}&interval={timeType}"
    response = requests.request("GET", url, headers=cg_headers)
    return json.loads(response.text)

def get_lsur_data(symbol, Timeframe="5m"):
    timeType = candles[Timeframe]
    url = f"https://open-api.coinglass.com/api/pro/v1/futures/longShort_chart?symbol={symbol}&interval={timeType}"
    response = requests.request("GET", url, headers=cg_headers)
    return json.loads(response.text)

def get_lsur_binance(symbol, timeframe="5m", limit=24):
    url = f"https://www.binance.com/futures/data/globalLongShortAccountRatio?symbol={symbol}USDT&period={timeframe}&limit={limit}"
    response = requests.request("GET", url, headers=cg_headers)
    return json.loads(response.text)

def get_lsur_binance_top_position(symbol, timeframe="5m", limit=24):
    #Top Trader Long/Short Ratio (Position)
    #幣安大戶多空比倉位
    url = f"https://www.binance.com/futures/data/topLongShortPositionRatio?symbol={symbol}USDT&period={timeframe}&limit={limit}"
    response = requests.request("GET", url, headers=cg_headers)
    return json.loads(response.text)

def get_lsur_binance_top_account(symbol, timeframe="5m", limit=24):
    #Top Trader Long/Short Ratio (Accounts)
    #幣安大戶多空比帳戶
    url = f"https://www.binance.com/futures/data/topLongShortAccountRatio?symbol={symbol}USDT&period={timeframe}&limit={limit}"
    response = requests.request("GET", url, headers=cg_headers)
    return json.loads(response.text)

def get_liq_detial(symbol, Timeframe="5m"):
    timeType = candles[Timeframe]
    url = f"https://open-api.coinglass.com/api/pro/v1/futures/liquidation/detail/chart?symbol={symbol}&timeType={timeType}"
    response = requests.request("GET", url, headers=cg_headers)
    return json.loads(response.text)

def get_fr(symbol, ttype="U"):
    url = f"https://open-api.coinglass.com/api/pro/v1/futures/funding_rates_chart?symbol={symbol}&type={ttype}"
    response = requests.request("GET", url, headers=cg_headers)
    return json.loads(response.text)

def send_to_bot(tid_, msg, bot_token_):
    url = f'https://api.telegram.org/bot{bot_token_}/sendMessage'
    headers = {'content-type': 'application/json'}
    data  = f'{{"chat_id": "{tid_}", "text": "{msg}", "parse_mode": "HTML", "disable_web_page_preview": true}}'.encode('utf-8')
    r = requests.post(url, data=data, headers=headers, verify=False)

In [168]:
#畫K線
def plotly_kline(data):
    candlestick = px.Candlestick(x=data["ohlc_df"].index,
                    open=data["ohlc_df"]['Open'],
                    high=data["ohlc_df"]['High'],
                    low=data["ohlc_df"]['Low'],
                    close=data["ohlc_df"]['Close'],name="Kline")

    liq_sell    = px.Bar(x=data["ohlc_df"].index,
                    y=data["ohlc_df"]['allnet_liq_sell'],
                    name = "sell",
                    marker_color = "blue")

    liq_buy     = px.Bar(x=data["ohlc_df"].index,
                    y=data["ohlc_df"]['allnet_liq_buy']  * -1,
                    name="buy",
                    marker_color="red" )

    wt1         = px.Scatter(x=data["ohlc_df"].index,
                    y=data["ohlc_df"]['wt1'],
                    line=dict(color='orange', width=1),
                    mode='lines',name=f"wt1")       

    wt2         = px.Scatter(x=data["ohlc_df"].index,
                    y=data["ohlc_df"]['wt2'],
                    line=dict(color='green', width=1),
                    mode='lines',name=f"wt2")                                                                                    
    
    title_text = f"{data['Exchanger']} timeframe:{data['Timeframe']} Last update: {data['ohlc_df'].index[-1]}"
    

    fig = make_subplots(rows=3, cols=1, shared_xaxes=True,row_heights=[0.4,0.3,0.3],vertical_spacing=0.03)
    fig.add_traces([candlestick],rows=1,cols=1)
    fig.add_traces([liq_sell,liq_buy],rows=2,cols=1)
    fig.add_traces([wt1,wt2],rows=3,cols=1)
    fig.update_layout(xaxis_rangeslider_visible=False)
    fig.update_layout(title=title_text,autosize=False,width=1000,height=600)

    fig["layout"]["template"]="plotly_dark"

    fig.show()
    #fig.write_image(f"{data['symbol'].split('/')[0]}.png")

wavetrend
https://github.com/LongandShortio/WaveTrend-bot/commit/4172abfda951758b867054cfcecc818faa4b5e98#diff-6c0accc162afef14679665fb3d39f80e56206be2cc1b18984d0f53ce2a6a6e16

In [169]:
#wavetrend

import logging
import pandas as pd
import numpy as np
#log = logging.getLogger(__name__)


def HA( dataframe ):
    """
    Returns Heikin Ashi candles
    """

    df = dataframe.copy()

    df['HA_Close']=(df.Open + df.High + df.Low + df.Close)/4

    df.reset_index(inplace=True)

    ha_open = [ (df.Open[0] + df.Close[0]) / 2 ]
    [ ha_open.append((ha_open[i] + df.HA_Close.values[i]) / 2) \
    for i in range(0, len(df)-1) ]
    df['HA_Open'] = ha_open

    df.set_index('Time', inplace=True)

    df['HA_High']=df[['HA_Open','HA_Close','High']].max(axis=1)
    df['HA_Low']=df[['HA_Open','HA_Close','Low']].min(axis=1)

    return df



def wave_trend(df, n1=10, n2=21):
    """
    Computes the wave trend indicators
    """
    df_ha=HA(df)
    ap=(df_ha['HA_High']+df_ha['HA_Low']+df_ha['HA_Close'])/3
    esa=ap.ewm(span=n1, min_periods=n1).mean().dropna() #ema(ap, n1)
    d=abs(ap - esa).ewm(span=n1, min_periods=n1).mean().dropna() #ema(abs(ap - esa), n1)
    ci=(ap.iloc[-len(ap):] - esa) / (0.015 * d)
    tci =ci.ewm(span=n2, min_periods=n2).mean().dropna() #ema(ci, n2)
    wt1 = tci
    wt2 = wt1.rolling(4, min_periods=4).mean().dropna() #sma(wt1,4)

    df["wt1"] = wt1
    df["wt2"] = wt2


    diff=wt1.iloc[-len(wt2):]-wt2

    df["diff"] = diff
    diff= pd.DataFrame(diff, index=df.index, columns=['diff'])
    return df

In [170]:


exchange = ccxt.binance()
ohlc_data = exchange.fetch_ohlcv(symbol=f"{symbol_name}/USDT",timeframe="1h",limit=150)

cols=["Time","Open","High","Low","Close","Volume"]
ohlc_df = pd.DataFrame(ohlc_data,columns=cols,)

ohlc_df = wave_trend(ohlc_df)


liq_data = get_liq_detial(symbol=symbol_name,Timeframe="1h")
liq_data = liq_data["data"]
liq_list = list()
for i in liq_data:
    for j in i["list"]:
        if j["exchangeName"] == "Binance":
            binance_liq_sell = j["sellVolUsd"]
            binance_liq_buy = j["buyVolUsd"]
            break
        else:
            binance_liq_sell = 0
            binance_liq_buy = 0 

    liq_list.append({
        "Time": i["createTime"],
        "allnet_liq_sell": i["sellVolUsd"],
        "allnet_liq_buy":i["buyVolUsd"],
        "Binance_liq_sell": binance_liq_sell,
        "Binance_liq_buy": binance_liq_buy
        })

liq_df = pd.DataFrame(liq_list)    


ohlc_df = ohlc_df.merge(right=liq_df, left_on="Time",right_on="Time",how="outer")
#tzone = dt.timezone(dt.timedelta(hours=8))

ohlc_df["Time"] = pd.to_datetime(ohlc_df["Time"],unit="ms",utc=True).map(lambda x: x.tz_convert('Asia/Taipei'))
ohlc_df.set_index("Time",inplace=True)

ohlc_df = wave_trend(ohlc_df)

ta_data = {
    "Exchanger":"undefine",
    "Timeframe":"1h",
    "ohlc_df":ohlc_df
}


In [171]:
plotly_kline(ta_data)

In [1]:
xxx = get_oi_chart_data("AVAX","1h")
xxx

NameError: name 'get_oi_chart_data' is not defined